<h1 style='color:black'> Exception Handling </h1>

Exceptions are the way that python handle control flows.

In [4]:
import sys

# User defined exception. Very often, exceptions are derived from Exception.
class BloombergError(Exception):
    pass

li = []

try:
    raise BloombergError('server down')
    li[0] = 67
    10 / 0
except BloombergError: # All exceptions are classes, but there is no parantheses in the class name after except.
    exc_type, exc_obj, exc_tb = sys.exc_info() # Exception type, obj and traceback
    sys.stderr.write( str(exc_obj) + '\n' ) # Quite often in exception handling, the return type is an object, even though superficially it should be strings.
    raise Exception( 'Some more useful information for the outer layer of code' ) # re-raise the exception
    sys.exit(3)
except (ZeroDivisionError, IndexError): # Specify a tuple to share common handling between different errors - it has to be a tuple!!
    exc_type, exc_obj, exc_tb = sys.exc_info()
    sys.exit(1)
except Exception, value: # value provides object reference to the exception as well
    
    exc_type, exc_obj, exc_tb = sys.exc_info()
    sys.stderr.write( str(exc_type) + '\n' )
    sys.stderr.write( 'unexpected exception\n')
    sys.exit(1)
else: # The else block helps minimize the amount of code in try blocks and visually distinguish the success case from the try/except blocks
    print('No exceptions raised')
finally: # suitable for common cleanup, regardless of whether exceptions occur
    print('always done')


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-4-a0efea8a3416>, line 31)

To catch multiple exceptions in one `except`, you will need to specify a `tuple`; see the 3rd mistake in [this post](https://www.evernote.com/shard/s191/nl/21353936/06c8859d-90c6-4688-a5a9-ef37f71e38f2?title=The%2010%20Most%20Common%20Mistakes%20That%20Python%20Developers%20Make%20%7C%20Toptal).

The `try`-`except` statement has an optional `else` clause, which, when present, must follow all `except` clauses. It is useful for code that must be executed if the `try` **clause does not raise an exception**.

The use of the `else` clause is better than adding additional code to the `try` clause because it avoids accidentally catching an exception that wasn’t explicitly handled by the `except` statement: if instead it was in the `else` block, it will propagate up. 

If a `finally` clause is present, the `finally` clause will execute **as the last task before the try statement completes**. The `finally` clause **runs whether or not the try statement produces an exception**. The following points discuss more complex cases when an exception occurs:

- If an exception occurs during execution of the `try` clause, the exception may be handled by an `except` clause. If the exception is not handled by an `except` clause, **the exception is re-raised after** the `finally` clause has been executed.

- An exception could occur during execution of an `except` or `else` clause. Again, **the exception is re-raised after the finally clause has been executed**.

- If the `finally` clause executes a `break`, `continue` or `return` statement, **exceptions are not re-raised**.

- If the try statement reaches a `break`, `continue` or `return` statement, the `finally` clause will **execute just prior to** the `break`, `continue` or `return` statement’s execution.

- If a `finally` clause includes a return statement, the returned value will be the one from the `finally` clause’s return statement, not the value from the `try` clause’s return statement.

Building on the first point above, when exception is raised in handler, original exception can get lost. The right way is to perform a cleanup in a finally block; compare the two versions of `myfunc` below. 

In [ ]:
import traceback
import logging

def doit():
    # some nested function encounters a problem
    raise RuntimeError('original error')
    
def myfunc_v1():
    try:
        doit()
    except:
        # if errorCorrection() fails
        # it will mask out the original exception
        errorCorrection()
        
def myfunc_v2():
    try:
        doit()
    except:
        try: # re-raise the original error first
            raise
    finally:
        # try handling the problem locally
        try:
            errorCorrection()
        except: 
            traceback.print_exc() 
            # exactly mimics the behavior of the python interpreter when it prints a stack trace
            # print_exc takes two optional variables: limit and file - prints limit entries in a stack tracce to a file.
            # if omitted, the full trace is printed to the console.
            logging.error("Some error occurs")

The above also touches on two good practices concerning exceptions: <font color='red'> **tracebacks** </font> and <font color='red'> **logging** </font>

## References
- Item 13. [< Effective Python >](https://www.evernote.com/shard/s191/nl/21353936/80774c17-012b-c7d8-e425-a9b3eb0d55f3?title=Effective%20Python).